## Seach Enging with Tools and Agents

### In-built Tools
- Arxiv (Research Papers), 
- Wikipedia

### Tools Creation

In [2]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [4]:
## Used in-built tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [5]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [6]:
tools = [wiki, arxiv]

### Custom Tools

In [8]:
## Custom tools [RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f781c9de770>, search_kwargs={})

In [11]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about Langsmith")
retriever_tool.name

'langsmith-search'

In [20]:
tools = [wiki, arxiv,retriever_tool]

In [21]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/mnt/d/Online Learning/11. Complete Generative AI Course with Langchain and Huggingface/02. Langchain Projects/01-Q&A_Chatbot/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7f78538a8430>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddi

### Run all this tools with Agents and LLM Models

In [22]:
## Tools, LLM -->AgentExecutor

import os
from langchain_groq import ChatGroq
import openai
from dotenv import load_dotenv
load_dotenv

groq_api_key = os.getenv("GROQ_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(groq_api_key = groq_api_key, model_name = "Llama3-8b-8192")

### Prompt Template

In [23]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Agents

In [24]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

#### Agent Executer

In [25]:
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent=agent,tools = tools, verbose=True)

In [26]:
agent_executer.invoke({"input": "Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic ‚Äî score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with automatic version control and collaboration features.
LangSmith + LangChain OSSLangSmith is framework-agnostic ‚Äî¬†it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up Lan

{'input': 'Tell me about Langsmith',
 'output': "Based on the output from the Langsmith-search tool, it appears that Langsmith is a platform for building production-grade LLM (Large Language Model) applications. It allows users to closely monitor and evaluate their applications, enabling them to ship quickly and with confidence. Langsmith provides observability features, evaluation tools, and prompt engineering capabilities to help users develop and refine their LLM applications.\n\nLangsmith seems to be a useful tool for developers working with LLMs, particularly in terms of debugging and troubleshooting non-deterministic results. The platform offers a range of features, including tracing, metrics, dashboards, and alerts, to help users gain insights into their application's performance and behavior.\n\nAdditionally, Langsmith appears to be compatible with other frameworks and tools, such as LangChain and LangGraph, making it a versatile solution for building and deploying LLM applicat

In [27]:
agent_executer.invoke({"input": "What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witBased on the result from the tool call "call_dv4h", it seems that machine learning is a field of study that focuses on developing and studying statistical algorithms that can learn from data and generalize to unseen data. This technology enables machines to perform tasks that typically require human intelligence, such as image recognition, speech recognition, and natural language processing.

Would you like me to explore this topic further, or is there something specific you'd like to know about machine learning?

> Finished chain.


{'input': 'What is machine learning',
 'output': 'Based on the result from the tool call "call_dv4h", it seems that machine learning is a field of study that focuses on developing and studying statistical algorithms that can learn from data and generalize to unseen data. This technology enables machines to perform tasks that typically require human intelligence, such as image recognition, speech recognition, and natural language processing.\n\nWould you like me to explore this topic further, or is there something specific you\'d like to know about machine learning?'}

In [28]:
agent_executer.invoke({"input": "What is attention is the paper 1706.03762 all about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" is about introducing a new deep learning architecture called the transformer, which is capable of performing tasks such as machine translation, without relying on traditional recurrent neural network (RNN) or convolutional neural network (CNN) architectures. The transformer uses self-attention mech

{'input': 'What is attention is the paper 1706.03762 all about?',
 'output': 'The paper "Attention Is All You Need" is about introducing a new deep learning architecture called the transformer, which is capable of performing tasks such as machine translation, without relying on traditional recurrent neural network (RNN) or convolutional neural network (CNN) architectures. The transformer uses self-attention mechanisms to process input sequences in parallel, allowing it to handle long-range dependencies and achieve state-of-the-art results in various natural language processing tasks.'}